In [10]:
# 工程准备
print("复制工程到根目录")
#!cp -r /kaggle/input/rftproj/yolo8 /yolo8
!ls /yolo8 -a
!pip install -e /yolo8/

# 查看GPU是否存在：最高支持CUDA12.6
import torch
print("[0]GPU:")
def check_gpu_availability():
    # 使用 torch.cuda.is_available() 函数检测 GPU 是否可用
    if torch.cuda.is_available():
        # 获取可用 GPU 的数量
        num_gpus = torch.cuda.device_count()
        print(f"GPU 可用，可用的 GPU 数量为: {num_gpus}")
        # 遍历每个可用的 GPU 并打印其名称
        for i in range(num_gpus):
            gpu_name = torch.cuda.get_device_name(i)
            print(f"GPU {i}: {gpu_name}")
    else:
        print("GPU 不可用，将使用 CPU 进行计算。")
check_gpu_availability()
# 查看CUDA版本12.2
print("[1]CUDA:")
!nvcc --version
# 查看pytorch版本2.5.1
print("[2]torch:"+torch.__version__)

# 查看python版本3.10
print("[3]Python:")
!python --version
# 测试已通过

复制工程到根目录
.		 examples     .pre-commit-config.yaml  task
..		 .git	      README.md		       tests
CITATION.cff	 .gitignore   README.zh-CN.md	       ultralytics
CONTRIBUTING.md  LICENSE      requirements.txt	       ultralytics.egg-info
datasets	 MANIFEST.in  runs		       yolov8n.pt
docker		 mkdocs.yml   setup.cfg
docs		 none.txt     setup.py
Obtaining file:///yolo8
  Preparing metadata (setup.py) ... done
  Attempting uninstall: ultralytics
    Found existing installation: ultralytics 8.0.29
    Uninstalling ultralytics-8.0.29:
      Successfully uninstalled ultralytics-8.0.29
  Running setup.py develop for ultralytics
[0]GPU:
GPU 可用，可用的 GPU 数量为: 1
GPU 0: Tesla P100-PCIE-16GB
[1]CUDA:
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0
[2]torch:2.5.1+cu121
[3]Python:
Python 3.10.12


In [13]:
# 转到工作目录
cd /yolo8

/yolo8


In [15]:
# 开始训练
from ultralytics import YOLO

# Load a model
model = YOLO("yolov8n.yaml")  # build a new model from scratch
#model = YOLO("yolov8n.pt")  # load a pretrained model (recommended for training)

# Use the model
results = model.train(data="./datasets/RFT.yaml", epochs=100,imgsz=416,device=0)  # train the model


                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.Conv                  [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.Conv                  [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.C2f                   [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.Conv                  [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.C2f                   [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.Conv                  [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.C2f                   [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics.nn.modules.Conv                  [128

In [22]:
# 结果转存
!zip -r /kaggle/working/train.zip /yolo8/runs/detect/train3

  adding: yolo8/runs/detect/train3/ (stored 0%)
  adding: yolo8/runs/detect/train3/val_batch1_pred.jpg (deflated 6%)
  adding: yolo8/runs/detect/train3/train_batch2.jpg (deflated 2%)
  adding: yolo8/runs/detect/train3/train_batch9451.jpg (deflated 8%)
  adding: yolo8/runs/detect/train3/val_batch2_pred.jpg (deflated 8%)
  adding: yolo8/runs/detect/train3/val_batch2_labels.jpg (deflated 8%)
  adding: yolo8/runs/detect/train3/P_curve.png (deflated 5%)
  adding: yolo8/runs/detect/train3/events.out.tfevents.1739000254.298fead7a3c3.31.0 (deflated 72%)
  adding: yolo8/runs/detect/train3/weights/ (stored 0%)
  adding: yolo8/runs/detect/train3/weights/best.pt (deflated 11%)
  adding: yolo8/runs/detect/train3/weights/last.pt (deflated 11%)
  adding: yolo8/runs/detect/train3/F1_curve.png (deflated 4%)
  adding: yolo8/runs/detect/train3/results.csv (deflated 85%)
  adding: yolo8/runs/detect/train3/val_batch1_labels.jpg (deflated 7%)
  adding: yolo8/runs/detect/train3/args.yaml (deflated 51%)
  add